In [1]:
import pandas as pd
import ast, json
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer 
# Import Punkt
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\My Probook
[nltk_data]     G2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
tweets_df= pd.read_csv('old_tweets.csv')
tweets_df.head()

,tweet_count,city,body,date,hashtags,link,geo
0,0,New York,@UberEats Promo Code never works for 50%. what...,2020-07-14 23:57:38+00:00,NaN,https://twitter.com/FashionsWeek/status/128318...,NaN
1,1,New York,Yea they’re terrible smh. @UberEats @Uber_Support,2020-07-14 23:36:33+00:00,NaN,https://twitter.com/kensthetic_/status/1283183...,NaN
2,2,New York,@UberEats when are you coming to upstate ny?,2020-07-14 23:33:33+00:00,NaN,https://twitter.com/Thebobover/status/12831828...,NaN
3,3,New York,@diginn why does the dig inn app and @UberEats...,2020-07-14 23:30:41+00:00,NaN,https://twitter.com/Hot4TaterTots/status/12831...,NaN
4,4,New York,For my brother to have his Uber job back! He h...,2020-07-14 23:04:47+00:00,NaN,https://twitter.com/Rayofshine69/status/128317...,NaN


In [3]:
tweets_df.isnull().sum()

tweet_count       0
city              0
body              5
date              0
hashtags       2186
link              0
geo            3013
dtype: int64

In [4]:
tweets_df.dropna(subset=['body'], inplace=True)

In [5]:
tweets_df.isnull().sum()
print(tweets_df.shape)

(3008, 7)


In [6]:
tweets_df.drop_duplicates(subset ='body', 
                     keep = False, inplace = True) 
tweets_df.head()

,tweet_count,city,body,date,hashtags,link,geo
0,0,New York,@UberEats Promo Code never works for 50%. what...,2020-07-14 23:57:38+00:00,NaN,https://twitter.com/FashionsWeek/status/128318...,NaN
1,1,New York,Yea they’re terrible smh. @UberEats @Uber_Support,2020-07-14 23:36:33+00:00,NaN,https://twitter.com/kensthetic_/status/1283183...,NaN
2,2,New York,@UberEats when are you coming to upstate ny?,2020-07-14 23:33:33+00:00,NaN,https://twitter.com/Thebobover/status/12831828...,NaN
3,3,New York,@diginn why does the dig inn app and @UberEats...,2020-07-14 23:30:41+00:00,NaN,https://twitter.com/Hot4TaterTots/status/12831...,NaN
4,4,New York,For my brother to have his Uber job back! He h...,2020-07-14 23:04:47+00:00,NaN,https://twitter.com/Rayofshine69/status/128317...,NaN


In [9]:
print(tweets_df.shape)

(2975, 7)


# 1. Tokenizer + Lematizer

In [10]:
# Create function to tokenize the sentences
def tokenize_cleaner(x):
    x = str(x)
    mylist =  word_tokenize(x)
    return mylist

In [11]:
tweets_df['tokenized'] = tweets_df['body'].apply(tokenize_cleaner) 
tweets_df.head(2)

,tweet_count,city,body,date,hashtags,link,geo,tokenized
0,0,New York,@UberEats Promo Code never works for 50%. what...,2020-07-14 23:57:38+00:00,NaN,https://twitter.com/FashionsWeek/status/128318...,NaN,"[@, UberEats, Promo, Code, never, works, for, ..."
1,1,New York,Yea they’re terrible smh. @UberEats @Uber_Support,2020-07-14 23:36:33+00:00,NaN,https://twitter.com/kensthetic_/status/1283183...,NaN,"[Yea, they, ’, re, terrible, smh, ., @, UberEa..."


In [12]:
# wordnet is a lexical database for the English language that helps the script determine the base word. 
# You need the averaged_perceptron_tagger resource to determine the context of a word in a sentence
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to C:\Users\My Probook
[nltk_data]     G2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\My Probook G2\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# 2. Remove the noise

In [13]:
import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [14]:
tweets_df['cleaned'] = tweets_df['tokenized'].apply(remove_noise)
tweets_df.head(2)
print(tweets_df['cleaned'][0])

['ubereats', 'promo', 'code', 'never', 'work', 'for', '50', 'what', 'do', 'i', 'do']


In [15]:
# Download stopwords
# In addition to this, you will also remove stop words using a built-in set of stop words in NLTK, 
# which needs to be downloaded separately
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [16]:
stop_words = stopwords.words('english')

def remove_stopword(x):
    return [word for word in x if word not in stop_words]

tweets_df['cleaned'] = tweets_df['cleaned'].apply(lambda x:remove_stopword(x))

In [17]:
tweets_df['cleaned'][0]

['ubereats', 'promo', 'code', 'never', 'work', '50']

In [18]:
# tweets_df.to_csv('UberEats.csv',index=False)